In [2]:
from modelarts import workflow as wf

home_path = wf.data.Storage(name='workspace', title='workflow_workspace')


def get_training_job_spec():
    return wf.steps.JobSpec(
        resource=wf.steps.JobResource(
            flavor=wf.Placeholder(
                name='training_flavor',
                placeholder_type=wf.PlaceholderType.JSON,
                description='训练节点规格'
            ),
            node_count=wf.Placeholder(name='training_node_count', placeholder_type=wf.PlaceholderType.INT, default=1,
                                 description='训练节点个数')
        )
    )

def process_data(home_path, depend_steps=None):
    training_job_code_path = home_path.join('code/')
    training_job_boot_file = home_path.join('code/data_process.py')

    return wf.steps.JobStep(
        name='process_data',
        title='process_data',
        algorithm=wf.BaseAlgorithm(
            code_dir=training_job_code_path,
            boot_file=training_job_boot_file,
            engine=wf.steps.JobEngine(engine_name='TensorFlow', engine_version='tensorflow_2.1.0-cuda_10.1-py_3.7-ubuntu_18.04-x86_64'),
            parameters=[
                wf.AlgorithmParameters(name='input_path', value='obs://shp-cn4/wyh_test/pipeline_cicd/ml-1m/raw_data'),
                wf.AlgorithmParameters(name='output_path', value='obs://shp-cn4/wyh_test/pipeline_cicd/ml-1m/output')]
        ),
        depend_steps=depend_steps,
        spec=get_training_job_spec()
    )

generated new fontManager


In [3]:


def train_model(home_path, depend_steps=None):
    # 代码路径
    training_job_code_path = home_path.join('code/')
    training_job_boot_file = home_path.join('code/train_model.py')

    return wf.steps.JobStep(
        name='train_model',
        title='train_model',
        algorithm=wf.BaseAlgorithm(
            code_dir=training_job_code_path,
            boot_file=training_job_boot_file,
            engine=wf.steps.JobEngine(engine_name='TensorFlow', engine_version='tensorflow_2.1.0-cuda_10.1-py_3.7-ubuntu_18.04-x86_64'),
            parameters=[
                wf.AlgorithmParameters(name='input_path', value='obs://shp-cn4/wyh_test/pipeline_cicd/ml-1m/output'),
                wf.AlgorithmParameters(name='output_path', value='obs://shp-cn4/wyh_test/pipeline_cicd/ml-1m/output')]
        ),
        depend_steps=depend_steps,
        spec=get_training_job_spec()
    )

In [4]:

process_data_job = process_data(home_path)
train_model_job = train_model(home_path, depend_steps=[process_data_job])

workflow = wf.Workflow(
    name='ml-1m-mlops',
    desc='ml-1m-mlops',
    steps=[
        process_data_job,
        train_model_job
    ],
    storages=[home_path])

workflow.release()

start releasing Workflow ml-1m-mlops
Workflow ml-1m-mlops successfully released, and workflow ID is 148dc954-7c4f-479b-8fd5-2888434dfdd7


'148dc954-7c4f-479b-8fd5-2888434dfdd7'

In [1]:
from modelarts.session import Session
sess = Session()
sess.obs.copy("/home/ma-user/work/ml-1m", "obs://shp-cn4/wyh_test/pipeline_cicd")

/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
100%|██████████| 11/11 [00:01<00:00,  9.50it/s]
